### 1. data loader 

In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np


from keras.preprocessing.image import img_to_array
class loader:
    def __init__(self):
        self.x_data=[]
        self.y_data=[]

    def load(self,path):
        files=os.listdir(path)
        for i in files:
            img=cv2.imread(path+'/'+i,cv2.IMREAD_GRAYSCALE)# 400 x 600
            img=cv2.resize(img,(240,240))
            #img=cv2.resize(img,(240,240))
            #print(img.shape)
            self.y_data.append(img_to_array(img))
            self.x_data.append(img_to_array(img))
        return self.x_data,self.y_data



img=cv2.imread('../input/grid-map/111202511.png',cv2.IMREAD_GRAYSCALE)# 400 x 600
img=cv2.resize(img,(240,240)) # 宽 x 高
plt.imshow(img, cmap=plt.cm.gray)
plt.xticks([])  # 去掉x轴
plt.yticks([])  # 去掉y轴
plt.axis('off')  # 去掉坐标轴
plt.show()

In [2]:
from keras.models import Model
from keras.layers import Conv2D,UpSampling2D,MaxPooling2D
from keras.layers import Input
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras import regularizers

class Models:
    def __init__(self,w,h,c):
        self.w=w
        self.h=h
        self.c=c
    def Arch1(self):
        inp=Input(shape=(self.w,self.h,self.c))
        enc=Conv2D(64,(3,3),padding='same')(inp)
        enc=BatchNormalization()(enc)
        enc=LeakyReLU(alpha=0.1)(enc)
        enc=MaxPooling2D(pool_size=(2,2))(enc)
        enc=Conv2D(32,(3,3),padding='same')(enc)
        enc=LeakyReLU(alpha=0.1)(enc)
        enc=BatchNormalization()(enc)
        enc=MaxPooling2D(pool_size=(2,2))(enc)
        enc=Conv2D(16,(3,3),padding='same')(enc)
        enc=LeakyReLU(alpha=0.1)(enc)
        enc=BatchNormalization()(enc)
        enc=MaxPooling2D(pool_size=(2,2))(enc)
        enc=Conv2D(8,(3,3),padding='same')(enc)
        enc=LeakyReLU(alpha=0.1)(enc)
        enc=MaxPooling2D(pool_size=(2,2))(enc)


        dec=Conv2D(8,(3,3),padding='same')(enc)
        dec=LeakyReLU(alpha=0.1)(dec)
        dec=UpSampling2D((2,2))(dec)
        dec=Conv2D(16,(3,3),padding='same')(dec)
        dec=LeakyReLU(alpha=0.1)(dec)
        dec=UpSampling2D((2,2))(dec)
        dec=Conv2D(32,(3,3),padding='same')(dec)
        dec=LeakyReLU(alpha=0.1)(dec)
        dec=UpSampling2D((2,2))(dec)
        dec=Conv2D(64,(3,3),padding='same')(dec)
        dec=LeakyReLU(alpha=0.1)(dec)
        dec=UpSampling2D((2,2))(dec)
        final=Conv2D(1,(3,3),padding='same',activation='sigmoid')(dec)
        auto=Model(inp,final)
        return auto

    def Arch2(self):
        inp=Input(shape=(self.w,self.h,self.c))
        enc=Conv2D(64,(3,3),padding='same',activity_regularizer=regularizers.l1(10e-5))(inp)
        enc=BatchNormalization()(enc)
        enc=LeakyReLU(alpha=0.1)(enc)
        enc=MaxPooling2D(pool_size=(2,2))(enc)
        enc=Conv2D(32,(3,3),padding='same',activity_regularizer=regularizers.l1(10e-5))(enc)
        enc=LeakyReLU(alpha=0.1)(enc)
        enc=BatchNormalization()(enc)
        enc=MaxPooling2D(pool_size=(2,2))(enc)
        enc=Conv2D(16,(3,3),padding='same',activity_regularizer=regularizers.l1(10e-5))(enc)
        enc=LeakyReLU(alpha=0.1)(enc)
        enc=BatchNormalization()(enc)
        enc=MaxPooling2D(pool_size=(2,2),activity_regularizer=regularizers.l1(10e-5))(enc)
        enc=Conv2D(8,(3,3),padding='same')(enc)
        enc=LeakyReLU(alpha=0.1)(enc)
        enc=MaxPooling2D(pool_size=(2,2))(enc)


        dec=Conv2D(8,(3,3),padding='same',activity_regularizer=regularizers.l1(10e-5))(enc)
        dec=LeakyReLU(alpha=0.1)(dec)
        dec=UpSampling2D((2,2))(dec)
        dec=Conv2D(16,(3,3),padding='same',activity_regularizer=regularizers.l1(10e-5))(dec)
        dec=LeakyReLU(alpha=0.1)(dec)
        dec=UpSampling2D((2,2))(dec)
        dec=Conv2D(32,(3,3),padding='same',activity_regularizer=regularizers.l1(10e-5))(dec)
        dec=LeakyReLU(alpha=0.1)(dec)
        dec=UpSampling2D((2,2))(dec)
        dec=Conv2D(64,(3,3),padding='same',activity_regularizer=regularizers.l1(10e-5))(dec)
        dec=LeakyReLU(alpha=0.1)(dec)
        dec=UpSampling2D((2,2))(dec)
        final=Conv2D(1,(3,3),padding='same',activation='sigmoid')(dec)
        auto=Model(inp,final)
        return auto


## main 函数：

In [3]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import adam_v2

w=240
h=240
c=1

## 数据载入、训练集和测试集划分
load_img=loader()
x_data,y_data=load_img.load('../input/grid-map')
x_data=np.array(x_data,dtype='float')/255.0
y_data=np.array(y_data,dtype='float')/255.0
train_x,test_x,train_y,test_y=train_test_split(x_data,y_data,test_size=0.1,random_state=30)

In [4]:
## 模型载入
model=Models(w,h,c)
auto_encoder=model.Arch2()
auto_encoder.summary()

opt= adam_v2.Adam(learning_rate=0.001,decay=0.001/100)

auto_encoder.compile(optimizer= opt,loss='binary_crossentropy')
auto_encoder.fit(train_x,train_y,batch_size=4,shuffle='true',epochs=20,validation_data=(test_x,test_y),verbose=1)
auto_encoder.save_weights('my_model_weights.h5')


2022-05-06 12:04:14.263710: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 12:04:14.365011: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 12:04:14.366180: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 12:04:14.368120: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 240, 240, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 240, 240, 64)      640       
_________________________________________________________________
batch_normalization (BatchNo (None, 240, 240, 64)      256       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 240, 240, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 120, 120, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 120, 120, 32)      18464     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 120, 120, 32)      0     

2022-05-06 12:04:18.463484: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2022-05-06 12:04:20.345195: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


429/429 [==============================] - 16s 20ms/step - loss: 6.9583 - val_loss: 2.1708
Epoch 2/20
429/429 [==============================] - 8s 19ms/step - loss: 3.0385 - val_loss: 2.5787
Epoch 3/20
429/429 [==============================] - 8s 19ms/step - loss: 2.3433 - val_loss: 2.1195
Epoch 4/20
429/429 [==============================] - 8s 19ms/step - loss: 1.8034 - val_loss: 1.5601
Epoch 5/20
429/429 [==============================] - 8s 19ms/step - loss: 1.4470 - val_loss: 1.3413
Epoch 6/20
429/429 [==============================] - 8s 19ms/step - loss: 1.2699 - val_loss: 1.2316
Epoch 7/20
429/429 [==============================] - 8s 19ms/step - loss: 1.1704 - val_loss: 1.1504
Epoch 8/20
429/429 [==============================] - 8s 19ms/step - loss: 1.1188 - val_loss: 1.0973
Epoch 9/20
429/429 [==============================] - 8s 19ms/step - loss: 1.0847 - val_loss: 1.0125
Epoch 10/20
429/429 [==============================] - 8s 19ms/step - loss: 1.0621 - val_loss: 1.0199